In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler 
import math
from keras.callbacks import LearningRateScheduler

In [ ]:
from keras.optimizers import SGD

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ionosphere/ionosphere_data.csv")

In [ ]:
data.shape

In [ ]:
data.head(10)

In [ ]:
X=data.iloc[:,:-1].values

In [ ]:
X

In [ ]:
y=data.iloc[:,-1].values

In [ ]:
y

In [ ]:
seed=7
np.random.seed(seed)

In [ ]:
encoder = LabelEncoder() 
encoder.fit(y) 
Y= encoder.transform(y)

In [ ]:
epochs = 50
def baseline_model(): 
    # create model 
    model = Sequential() 
    model.add(Dense(34, input_dim=34, activation= "relu" )) 
    model.add(Dense(16, activation= "relu" ))
    model.add(Dense(1, activation= "sigmoid" )) 
    # Compile model
     
    learning_rate = 0.1 
    model.compile(loss= "binary_crossentropy" , optimizer=sgd, metrics=[ "accuracy" ]) 
    return model

In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=epochs, batch_size=1, verbose=0)

In [ ]:
model = Sequential() 
model.add(Dense(34, input_dim=34,activation= "relu" )) 
model.add(Dense(16,activation= "relu" ))
model.add(Dense(1, activation= "sigmoid" )) 
# Compile model     
learning_rate = 0.1 
model.compile(loss= "binary_crossentropy" , optimizer=sgd, metrics=[ "accuracy" ])
results=model.fit(X, Y, validation_split=0.33,epochs=epochs, batch_size=28)
results

In [ ]:
def step_decay(epoch): 
    initial_lrate = 0.1 
    drop = 0.5 
    epochs_drop = 5.0 
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop)) 
    return lrate

In [ ]:
# learning schedule callback 
lrate = LearningRateScheduler(step_decay) 
callbacks_list = [lrate] 
# Fit the model 
model.fit(X, Y, validation_split=0.33, batch_size=1, callbacks=callbacks_list, verbose=2) 


In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=False, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

In [ ]:
def tr_plot(tr_data):
    #Plot the training and validation data
    history=tr_data.history
    tacc=results.history['accuracy']
    tloss=results.history['loss']
    vacc=results.history['val_accuracy']
    vloss=results.history['val_loss']
    Epoch_count=len(tloss)
    Epochs=[]
    for i in range (0,Epoch_count):
        Epochs.append(i+1)
    index_loss=np.argmin(vloss)#  this is the epoch with the lowest validation loss
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    val_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1)
    vc_label='best epoch= '+ str(index_acc + 1)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,8))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1,val_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout
    #plt.style.use('fivethirtyeight')
    plt.show()


In [ ]:
tr_plot(results)